⚡ Intermédiaire | ⏱ 60 min | 🔑 Concepts : requests, GET/POST, auth, pagination, error handling

# API REST : Consommer des APIs avec Python

## Objectifs

À la fin de ce notebook, vous serez capable de :
- Comprendre les concepts REST (ressources, verbes HTTP, status codes)
- Faire des requêtes GET, POST, PUT, DELETE
- Gérer l'authentification (API keys, Bearer tokens)
- Implémenter la pagination
- Gérer les erreurs et les timeouts
- Optimiser avec requests.Session()
- Respecter les rate limits

## Prérequis

- Python 3.8+
- Bases de HTTP
- Compréhension du format JSON

## 1. API REST : Concepts

**REST (Representational State Transfer)** : architecture pour les APIs web.

### Concepts clés

**Ressources** : Entités manipulées (users, products, orders, etc.)
- URL : `/api/users`, `/api/products/123`

**Verbes HTTP** :
- `GET` : Lire des données (idempotent, sans effet de bord)
- `POST` : Créer une ressource
- `PUT` : Mettre à jour (remplacer) une ressource
- `PATCH` : Modifier partiellement une ressource
- `DELETE` : Supprimer une ressource

**Status codes HTTP** :
- `200 OK` : Succès
- `201 Created` : Ressource créée
- `204 No Content` : Succès sans contenu
- `400 Bad Request` : Requête invalide
- `401 Unauthorized` : Non authentifié
- `403 Forbidden` : Non autorisé
- `404 Not Found` : Ressource non trouvée
- `429 Too Many Requests` : Rate limit dépassé
- `500 Internal Server Error` : Erreur serveur

**Installation :**
```bash
pip install requests
```

In [ ]:
import requests
import json
import time
from typing import List, Dict, Optional
from pprint import pprint

print(f"Requests version : {requests.__version__}")

## 2. GET : Lire des Données

La méthode la plus courante pour récupérer des données.

Nous utiliserons **JSONPlaceholder**, une API REST publique pour les tests.

In [ ]:
# GET simple
url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url)

print("Status code:", response.status_code)
print("Headers:", dict(response.headers))
print("\nContenu (JSON):")
pprint(response.json())

# Accès aux données
data = response.json()
print(f"\nTitre: {data['title']}")
print(f"User ID: {data['userId']}")

In [ ]:
# GET avec paramètres de requête
url = "https://jsonplaceholder.typicode.com/posts"
params = {
    'userId': 1,
    '_limit': 5  # Limiter à 5 résultats
}

response = requests.get(url, params=params)
print(f"URL complète: {response.url}")
print(f"\nNombre de posts: {len(response.json())}")
print("\nPremier post:")
pprint(response.json()[0])

In [ ]:
# GET avec headers personnalisés
url = "https://jsonplaceholder.typicode.com/posts/1"
headers = {
    'User-Agent': 'MyPythonApp/1.0',
    'Accept': 'application/json'
}

response = requests.get(url, headers=headers)
print("Request headers envoyés:")
pprint(dict(response.request.headers))

## 3. Response : Traiter la Réponse

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url)

# Attributs de Response
print("=== Attributs Response ===")
print(f"Status code: {response.status_code}")
print(f"OK (2xx): {response.ok}")
print(f"Reason: {response.reason}")
print(f"Encoding: {response.encoding}")
print(f"Content-Type: {response.headers.get('Content-Type')}")

# Contenu
print("\n=== Contenu ===")
print(f"text (str): {response.text[:100]}...")
print(f"content (bytes): {response.content[:100]}...")
print(f"json() (dict): {type(response.json())}")

# Temps de réponse
print(f"\nTemps de réponse: {response.elapsed.total_seconds():.3f}s")

# URL finale (après redirections)
print(f"URL finale: {response.url}")

## 4. POST : Créer des Données

Envoyer des données au serveur.

In [ ]:
# POST avec JSON
url = "https://jsonplaceholder.typicode.com/posts"
data = {
    'title': 'Mon nouveau post',
    'body': 'Contenu du post créé depuis Python',
    'userId': 1
}

response = requests.post(url, json=data)

print(f"Status code: {response.status_code}")
print("\nRéponse:")
pprint(response.json())

# Vérifier la création
if response.status_code == 201:
    print("\n✅ Ressource créée avec succès")
    created_id = response.json()['id']
    print(f"ID créé: {created_id}")

In [ ]:
# POST avec form data (application/x-www-form-urlencoded)
url = "https://jsonplaceholder.typicode.com/posts"
form_data = {
    'title': 'Post avec form data',
    'body': 'Corps du message',
    'userId': 1
}

response = requests.post(url, data=form_data)
print("Content-Type envoyé:", response.request.headers['Content-Type'])
print("\nRéponse:")
pprint(response.json())

## 5. PUT, PATCH, DELETE

Mettre à jour et supprimer des ressources.

In [ ]:
# PUT : remplacement complet
url = "https://jsonplaceholder.typicode.com/posts/1"
data = {
    'id': 1,
    'title': 'Post mis à jour',
    'body': 'Nouveau contenu complet',
    'userId': 1
}

response = requests.put(url, json=data)
print("PUT - Status:", response.status_code)
print("Réponse:")
pprint(response.json())

In [ ]:
# PATCH : modification partielle
url = "https://jsonplaceholder.typicode.com/posts/1"
data = {
    'title': 'Titre modifié'  # Seulement le titre
}

response = requests.patch(url, json=data)
print("PATCH - Status:", response.status_code)
print("Réponse:")
pprint(response.json())

In [ ]:
# DELETE : suppression
url = "https://jsonplaceholder.typicode.com/posts/1"

response = requests.delete(url)
print("DELETE - Status:", response.status_code)
print("Réponse:", response.json())

if response.status_code == 200:
    print("\n✅ Ressource supprimée")

## 6. Authentification

Différentes méthodes pour s'authentifier.

In [ ]:
# API Key dans l'URL (query parameter)
# NOTE: Éviter cette méthode (les URLs peuvent être loguées)
url = "https://api.example.com/data"
params = {'api_key': 'votre-cle-secrete'}
# response = requests.get(url, params=params)

print("❌ API Key dans URL : À ÉVITER")
print("Les URLs sont souvent loguées, exposant la clé\n")

In [ ]:
# API Key dans headers (recommandé)
url = "https://api.example.com/data"
headers = {
    'X-API-Key': 'votre-cle-secrete'
}
# response = requests.get(url, headers=headers)

print("✅ API Key dans headers : RECOMMANDÉ")
print("Headers:")
pprint(headers)

In [ ]:
# Bearer Token (JWT, OAuth)
url = "https://api.example.com/protected"
token = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9..."
headers = {
    'Authorization': f'Bearer {token}'
}
# response = requests.get(url, headers=headers)

print("✅ Bearer Token : Pour APIs modernes")
print("Headers:")
pprint(headers)

In [ ]:
# Basic Authentication
url = "https://api.example.com/basic"
response_demo = requests.get(
    url, 
    auth=('username', 'password'),
    timeout=5
)

# Ou manuellement
import base64
credentials = base64.b64encode(b'username:password').decode('ascii')
headers = {
    'Authorization': f'Basic {credentials}'
}
# response = requests.get(url, headers=headers)

print("✅ Basic Auth : Simple mais moins sécurisé")
print("À utiliser uniquement avec HTTPS")

## 7. Pagination

Récupérer de grandes quantités de données page par page.

In [ ]:
# Pagination par offset/limit
def fetch_all_posts_offset(base_url: str, limit: int = 10) -> List[Dict]:
    """
    Récupère tous les posts avec pagination offset/limit.
    """
    all_posts = []
    offset = 0
    
    while True:
        print(f"Fetching page (offset={offset}, limit={limit})...")
        
        response = requests.get(
            base_url,
            params={'_start': offset, '_limit': limit},
            timeout=10
        )
        response.raise_for_status()
        
        posts = response.json()
        
        if not posts:
            break
        
        all_posts.extend(posts)
        offset += limit
        
        # Éviter de surcharger l'API
        time.sleep(0.1)
    
    return all_posts

# Test
url = "https://jsonplaceholder.typicode.com/posts"
posts = fetch_all_posts_offset(url, limit=20)
print(f"\nTotal posts récupérés: {len(posts)}")

In [ ]:
# Pagination par numéro de page
def fetch_all_comments_page(base_url: str, per_page: int = 20) -> List[Dict]:
    """
    Récupère tous les commentaires avec pagination par page.
    """
    all_comments = []
    page = 1
    
    while True:
        print(f"Fetching page {page}...")
        
        response = requests.get(
            base_url,
            params={'_page': page, '_limit': per_page},
            timeout=10
        )
        response.raise_for_status()
        
        comments = response.json()
        
        if not comments:
            break
        
        all_comments.extend(comments)
        page += 1
        
        # Max 5 pages pour la démo
        if page > 5:
            print("Limite de 5 pages atteinte (démo)")
            break
        
        time.sleep(0.1)
    
    return all_comments

# Test
url = "https://jsonplaceholder.typicode.com/comments"
comments = fetch_all_comments_page(url, per_page=50)
print(f"\nTotal comments récupérés: {len(comments)}")

In [ ]:
# Pagination avec Link header (GitHub style)
def parse_link_header(link_header: str) -> Dict[str, str]:
    """
    Parse le Link header pour extraire les URLs de pagination.
    Exemple: <https://api.github.com/users?page=2>; rel="next"
    """
    links = {}
    if not link_header:
        return links
    
    for link in link_header.split(','):
        parts = link.split(';')
        if len(parts) == 2:
            url = parts[0].strip()[1:-1]  # Retirer < >
            rel = parts[1].strip().split('=')[1][1:-1]  # Retirer ""
            links[rel] = url
    
    return links

# Exemple
link_header = '<https://api.github.com/users?page=2>; rel="next", <https://api.github.com/users?page=5>; rel="last"'
links = parse_link_header(link_header)
print("Links extraits:")
pprint(links)

## 8. Gestion d'Erreurs

Gérer les erreurs HTTP et réseau de manière robuste.

In [ ]:
# raise_for_status() : lever une exception si erreur HTTP
url = "https://jsonplaceholder.typicode.com/posts/9999"

try:
    response = requests.get(url, timeout=5)
    response.raise_for_status()  # Lève HTTPError si 4xx ou 5xx
    print("Succès")
except requests.exceptions.HTTPError as e:
    print(f"❌ HTTP Error: {e}")
    print(f"Status code: {e.response.status_code}")
    print(f"Response: {e.response.text}")

In [ ]:
# Gestion complète des erreurs
def fetch_with_error_handling(url: str, **kwargs) -> Optional[Dict]:
    """
    Fetch avec gestion complète des erreurs.
    """
    try:
        response = requests.get(url, **kwargs)
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error: {e.response.status_code} - {e.response.reason}")
        return None
    
    except requests.exceptions.ConnectionError:
        print("❌ Connection Error: Impossible de se connecter au serveur")
        return None
    
    except requests.exceptions.Timeout:
        print("❌ Timeout: Le serveur n'a pas répondu à temps")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Request Error: {e}")
        return None
    
    except ValueError:
        print("❌ JSON Error: Impossible de parser la réponse")
        return None

# Tests
print("=== Tests gestion d'erreurs ===")

# Succès
data = fetch_with_error_handling(
    "https://jsonplaceholder.typicode.com/posts/1",
    timeout=5
)
if data:
    print(f"\n✅ Succès: {data['title']}")

# 404
print("\n--- Test 404 ---")
data = fetch_with_error_handling(
    "https://jsonplaceholder.typicode.com/posts/9999",
    timeout=5
)

# Timeout
print("\n--- Test Timeout ---")
data = fetch_with_error_handling(
    "https://httpbin.org/delay/10",
    timeout=1
)

## 9. Timeout : Toujours Spécifier un Timeout

**IMPORTANT** : Ne jamais oublier le timeout, sinon la requête peut pendre indéfiniment.

In [ ]:
# ❌ Sans timeout (DANGEREUX)
# response = requests.get(url)  # Peut pendre indéfiniment

# ✅ Avec timeout
url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url, timeout=5)  # 5 secondes max
print("✅ Requête avec timeout de 5s")

# Timeout séparé : (connect, read)
response = requests.get(url, timeout=(3, 10))
print("✅ Timeout: 3s pour se connecter, 10s pour lire")

# Recommandations
print("\n💡 Recommandations timeout:")
print("  - APIs rapides: 5-10s")
print("  - APIs lentes: 30-60s")
print("  - Téléchargements: timeout=(10, 300)")

## 10. Rate Limiting : Respecter les Limites

Ne pas surcharger les APIs avec trop de requêtes.

In [ ]:
import time

# Simple delay entre requêtes
def fetch_multiple_with_delay(urls: List[str], delay: float = 1.0) -> List[Dict]:
    """
    Fetch plusieurs URLs avec délai entre chaque.
    """
    results = []
    
    for i, url in enumerate(urls, 1):
        print(f"Fetching {i}/{len(urls)}: {url}")
        
        response = requests.get(url, timeout=10)
        if response.ok:
            results.append(response.json())
        
        # Attendre avant la prochaine requête
        if i < len(urls):
            time.sleep(delay)
    
    return results

# Test
urls = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://jsonplaceholder.typicode.com/posts/2",
    "https://jsonplaceholder.typicode.com/posts/3"
]

start = time.time()
results = fetch_multiple_with_delay(urls, delay=0.5)
elapsed = time.time() - start

print(f"\nRésultats: {len(results)}")
print(f"Temps total: {elapsed:.2f}s")

In [ ]:
# Backoff exponentiel en cas d'erreur
def fetch_with_retry(url: str, max_retries: int = 3) -> Optional[Dict]:
    """
    Fetch avec retry et backoff exponentiel.
    """
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            
            if response.status_code == 429:  # Rate limit
                retry_after = int(response.headers.get('Retry-After', 60))
                print(f"Rate limit atteint. Attente de {retry_after}s...")
                time.sleep(retry_after)
                continue
            
            response.raise_for_status()
            return response.json()
        
        except requests.exceptions.RequestException as e:
            wait_time = 2 ** attempt  # 1s, 2s, 4s, 8s...
            print(f"Erreur (tentative {attempt + 1}/{max_retries}): {e}")
            
            if attempt < max_retries - 1:
                print(f"Nouvelle tentative dans {wait_time}s...")
                time.sleep(wait_time)
            else:
                print("Max retries atteint")
                return None
    
    return None

# Test
data = fetch_with_retry("https://jsonplaceholder.typicode.com/posts/1")
if data:
    print(f"\n✅ Succès: {data['title']}")

## 11. Session : Réutiliser les Connexions

Optimiser les performances avec requests.Session().

In [ ]:
# Sans Session : nouvelle connexion à chaque requête
url = "https://jsonplaceholder.typicode.com/posts/1"

start = time.time()
for _ in range(5):
    response = requests.get(url, timeout=5)
time_without_session = time.time() - start

print(f"Sans Session (5 requêtes): {time_without_session:.3f}s")

In [ ]:
# Avec Session : connexion réutilisée
url = "https://jsonplaceholder.typicode.com/posts/1"

start = time.time()
with requests.Session() as session:
    # Configuration globale
    session.headers.update({
        'User-Agent': 'MyApp/1.0',
        'Accept': 'application/json'
    })
    
    for _ in range(5):
        response = session.get(url, timeout=5)

time_with_session = time.time() - start

print(f"Avec Session (5 requêtes): {time_with_session:.3f}s")
print(f"Gain: {time_without_session/time_with_session:.1f}x plus rapide")

In [ ]:
# Session avec configuration complète
class APIClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'X-API-Key': api_key,
            'User-Agent': 'MyAPIClient/1.0',
            'Accept': 'application/json'
        })
    
    def get(self, endpoint: str, **kwargs) -> Optional[Dict]:
        url = f"{self.base_url}/{endpoint}"
        kwargs.setdefault('timeout', 10)
        
        try:
            response = self.session.get(url, **kwargs)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Erreur: {e}")
            return None
    
    def close(self):
        self.session.close()

# Utilisation
client = APIClient(
    base_url="https://jsonplaceholder.typicode.com",
    api_key="demo-key"
)

post = client.get("posts/1")
if post:
    print(f"Post récupéré: {post['title']}")

client.close()
print("\n✅ Client fermé")

## 12. Exemple Complet : Pipeline de Données

Récupérer des données d'une API et les transformer en DataFrame.

In [ ]:
import pandas as pd
from typing import List, Dict

def fetch_all_users() -> List[Dict]:
    """
    Récupère tous les utilisateurs depuis l'API.
    """
    url = "https://jsonplaceholder.typicode.com/users"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur: {e}")
        return []

def transform_users(users: List[Dict]) -> pd.DataFrame:
    """
    Transforme les données utilisateurs en DataFrame.
    """
    # Aplatir la structure imbriquée
    transformed = []
    for user in users:
        transformed.append({
            'id': user['id'],
            'name': user['name'],
            'username': user['username'],
            'email': user['email'],
            'city': user['address']['city'],
            'company': user['company']['name']
        })
    
    return pd.DataFrame(transformed)

def fetch_user_posts(user_id: int) -> List[Dict]:
    """
    Récupère les posts d'un utilisateur.
    """
    url = "https://jsonplaceholder.typicode.com/posts"
    params = {'userId': user_id}
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur pour user {user_id}: {e}")
        return []

# Pipeline complet
print("=== Pipeline de Données ===")

# 1. Récupérer les utilisateurs
print("\n1. Récupération des utilisateurs...")
users = fetch_all_users()
print(f"Utilisateurs récupérés: {len(users)}")

# 2. Transformer en DataFrame
print("\n2. Transformation...")
df_users = transform_users(users)
print(df_users.head())

# 3. Enrichir avec le nombre de posts
print("\n3. Enrichissement avec nb de posts...")
post_counts = []
for user_id in df_users['id']:
    posts = fetch_user_posts(user_id)
    post_counts.append(len(posts))
    time.sleep(0.1)  # Rate limiting

df_users['nb_posts'] = post_counts

# 4. Résultats
print("\n4. Résultats finaux:")
print(df_users[['name', 'city', 'company', 'nb_posts']])

# 5. Analyse
print("\n5. Analyse:")
print(f"Total posts: {df_users['nb_posts'].sum()}")
print(f"Moyenne posts/user: {df_users['nb_posts'].mean():.1f}")
print(f"\nTop 3 auteurs:")
print(df_users.nlargest(3, 'nb_posts')[['name', 'nb_posts']])

## 13. Pièges Courants

### Piège 1 : Pas de Timeout

In [ ]:
print("❌ DANGEREUX: requests.get(url)")
print("   → Peut pendre indéfiniment")
print("\n✅ BON: requests.get(url, timeout=10)")
print("   → Timeout après 10 secondes")
print("\n💡 Toujours spécifier un timeout !")

### Piège 2 : Ignorer les Status Codes

In [ ]:
# ❌ Sans vérification
response = requests.get("https://jsonplaceholder.typicode.com/posts/9999", timeout=5)
# data = response.json()  # Peut planter si erreur

print(f"❌ Sans vérification: status={response.status_code}")
print("   → Peut causer des erreurs silencieuses\n")

# ✅ Avec vérification
response = requests.get("https://jsonplaceholder.typicode.com/posts/9999", timeout=5)
if response.ok:
    data = response.json()
else:
    print(f"✅ Avec vérification: Erreur {response.status_code}")
    print("   → Gestion explicite des erreurs")

# Ou avec raise_for_status()
try:
    response = requests.get("https://jsonplaceholder.typicode.com/posts/9999", timeout=5)
    response.raise_for_status()
    data = response.json()
except requests.exceptions.HTTPError:
    print("\n✅ Avec raise_for_status(): Exception levée")

### Piège 3 : Secrets dans le Code

In [ ]:
import os

# ❌ DANGEREUX: API key en dur
# api_key = "sk-1234567890abcdef"  # NE JAMAIS FAIRE ÇA

# ✅ BON: depuis variable d'environnement
api_key = os.getenv('API_KEY', 'default-key-for-demo')

# ✅ BON: depuis fichier .env (avec python-dotenv)
# from dotenv import load_dotenv
# load_dotenv()
# api_key = os.getenv('API_KEY')

print("✅ Utiliser des variables d'environnement pour les secrets")
print("✅ Ajouter .env à .gitignore")
print("❌ Ne JAMAIS commit des secrets dans le code")

## 14. Mini-Exercices

### Exercice 1 : Fetcher des Données d'API

Utilisez l'API JSONPlaceholder pour :
1. Récupérer tous les albums
2. Créer un DataFrame avec id, userId, title
3. Compter le nombre d'albums par utilisateur
4. Afficher les 3 utilisateurs avec le plus d'albums

In [ ]:
# Votre code ici


### Exercice 2 : Pagination

Récupérez tous les commentaires (500 au total) avec :
1. Pagination par batch de 50
2. Delay de 0.2s entre chaque requête
3. Gestion d'erreurs avec try/except
4. Affichage de la progression
5. Création d'un DataFrame final

In [ ]:
# Votre code ici


### Exercice 3 : Gestion d'Erreurs Robuste

Créez une fonction `robust_fetch()` qui :
1. Prend une URL et des paramètres
2. Fait jusqu'à 3 tentatives avec backoff exponentiel
3. Gère tous les types d'erreurs (HTTP, connexion, timeout, JSON)
4. Retourne un dict avec : {"success": bool, "data": dict|None, "error": str|None}
5. Testez avec des URLs valides et invalides

In [ ]:
# Votre code ici


---

## Solutions des Exercices

### Solution Exercice 1

In [ ]:
import pandas as pd

# 1. Récupérer les albums
url = "https://jsonplaceholder.typicode.com/albums"
response = requests.get(url, timeout=10)
response.raise_for_status()
albums = response.json()
print(f"1. Albums récupérés: {len(albums)}")

# 2. DataFrame
df_albums = pd.DataFrame(albums)
df_albums = df_albums[['id', 'userId', 'title']]
print("\n2. DataFrame:")
print(df_albums.head())

# 3. Nombre d'albums par utilisateur
albums_per_user = df_albums.groupby('userId').size().reset_index(name='nb_albums')
print("\n3. Albums par utilisateur:")
print(albums_per_user)

# 4. Top 3
top_3 = albums_per_user.nlargest(3, 'nb_albums')
print("\n4. Top 3 utilisateurs:")
print(top_3)

### Solution Exercice 2

In [ ]:
import time

def fetch_all_comments_paginated():
    base_url = "https://jsonplaceholder.typicode.com/comments"
    all_comments = []
    page = 1
    per_page = 50
    
    while True:
        try:
            print(f"Fetching page {page}...")
            
            response = requests.get(
                base_url,
                params={'_page': page, '_limit': per_page},
                timeout=10
            )
            response.raise_for_status()
            
            comments = response.json()
            
            if not comments:
                break
            
            all_comments.extend(comments)
            print(f"  → {len(comments)} commentaires récupérés (total: {len(all_comments)})")
            
            page += 1
            time.sleep(0.2)  # Rate limiting
            
        except requests.exceptions.RequestException as e:
            print(f"Erreur page {page}: {e}")
            break
    
    return all_comments

# Exécution
comments = fetch_all_comments_paginated()
df_comments = pd.DataFrame(comments)

print(f"\nTotal commentaires: {len(df_comments)}")
print("\nAperçu:")
print(df_comments.head())

### Solution Exercice 3

In [ ]:
import time
from typing import Dict, Any

def robust_fetch(url: str, max_retries: int = 3, **kwargs) -> Dict[str, Any]:
    """
    Fetch robuste avec retry et gestion complète des erreurs.
    
    Returns:
        Dict avec: {"success": bool, "data": dict|None, "error": str|None}
    """
    kwargs.setdefault('timeout', 10)
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, **kwargs)
            response.raise_for_status()
            data = response.json()
            
            return {
                "success": True,
                "data": data,
                "error": None
            }
        
        except requests.exceptions.HTTPError as e:
            error = f"HTTP {e.response.status_code}: {e.response.reason}"
            if attempt == max_retries - 1:
                return {"success": False, "data": None, "error": error}
        
        except requests.exceptions.ConnectionError:
            error = "Connection error: Impossible de se connecter"
            if attempt == max_retries - 1:
                return {"success": False, "data": None, "error": error}
        
        except requests.exceptions.Timeout:
            error = "Timeout: Pas de réponse du serveur"
            if attempt == max_retries - 1:
                return {"success": False, "data": None, "error": error}
        
        except ValueError:
            error = "JSON error: Impossible de parser la réponse"
            return {"success": False, "data": None, "error": error}
        
        except requests.exceptions.RequestException as e:
            error = f"Request error: {str(e)}"
            if attempt == max_retries - 1:
                return {"success": False, "data": None, "error": error}
        
        # Backoff exponentiel
        if attempt < max_retries - 1:
            wait_time = 2 ** attempt
            print(f"Tentative {attempt + 1} échouée. Retry dans {wait_time}s...")
            time.sleep(wait_time)
    
    return {"success": False, "data": None, "error": "Max retries atteint"}

# Tests
print("=== Tests robust_fetch ===")

# Test 1: URL valide
print("\n1. URL valide:")
result = robust_fetch("https://jsonplaceholder.typicode.com/posts/1")
print(f"Success: {result['success']}")
if result['success']:
    print(f"Title: {result['data']['title']}")

# Test 2: 404
print("\n2. URL 404:")
result = robust_fetch("https://jsonplaceholder.typicode.com/posts/9999")
print(f"Success: {result['success']}")
print(f"Error: {result['error']}")

# Test 3: Timeout
print("\n3. Timeout:")
result = robust_fetch("https://httpbin.org/delay/10", timeout=1, max_retries=2)
print(f"Success: {result['success']}")
print(f"Error: {result['error']}")

# Test 4: URL invalide
print("\n4. URL invalide:")
result = robust_fetch("https://invalid-domain-that-does-not-exist-12345.com", max_retries=2)
print(f"Success: {result['success']}")
print(f"Error: {result['error']}")